In [31]:
#!g1.1
%pip install transformers
%pip install auto-gptq
%pip install scipy
%pip install accelerate
%pip install optimum

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [32]:
#!g1.1
%pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [33]:
#!g1.1
import torch
import transformers
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd
import numpy as np
import random
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained("TheBloke/OpenBuddy-Llama2-13B-v11.1-GPTQ")

In [35]:
#!g1.1
model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenBuddy-Llama2-13B-v11.1-GPTQ", device_map='cuda')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
CUDA extension not installed.
CUDA extension not installed.


In [37]:
#!g1.1
# Пример использования:
text = '''
Введение
1. Краткая характеристика Тамбовской области. Оценка достигнутых целей и анализ потенциала социально-экономического развития Тамбовской области
1.1. Краткая характеристика Тамбовской области
1.2. Оценка достигнутых целей и анализ потенциала социально-экономического развития Тамбовской области
2. Стратегическая цель и приоритеты долгосрочного развития Тамбовской области
3. Целевой сценарий развития Тамбовской области
4. Сроки и этапы реализации Стратегии, показатели достижения целей и ожидаемые результаты реализации Стратегии
5. Стратегические направления и задачи развития Тамбовской области
5.1. Социальная модернизация на базе создания конкурентной среды, инноваций и цифровизации социальной сферы
5.2. Обеспечение комфортной и здоровой жизни населения на основе удобной и доступной инфраструктуры, благоприятной для проживания окружающей среды
5.3. Новая индустриализация, переход к цифровой и постиндустриальной экономике
5.4. Реформирование государственного и муниципального управления, взаимодействия государства, бизнеса и общества, расширение самоуправления и демократизация
6. Информация о государственных программах и иных механизмах реализации Стратегии
7. Оценка финансовых ресурсов, необходимых для реализации Стратегии
Таблица. Оценка финансового обеспечения Стратегии
Приложение 1. ОСНОВНЫЕ ПОКАЗАТЕЛИ СОЦИАЛЬНО-ЭКОНОМИЧЕСКОГО РАЗВИТИЯ ТАМБОВСКОЙ ОБЛАСТИ ЗА 2010 - 2016 ГГ.
Приложение 2. СВОТ-АНАЛИЗ СОЦИАЛЬНО-ЭКОНОМИЧЕСКОГО РАЗВИТИЯ ТАМБОВСКОЙ ОБЛАСТИ
Приложение 3. ДИНАМИКА ЦЕЛЕВЫХ ПОКАЗАТЕЛЕЙ СОЦИАЛЬНО-ЭКОНОМИЧЕСКОГО РАЗВИТИЯ ТАМБОВСКОЙ ОБЛАСТИ ДО 2035 ГОДА
Приоритетное направление 1. Социальная модернизация на базе создания конкурентной среды, инноваций и цифровизации социальной сферы
Приоритетное направление 2. Обеспечение комфортной и здоровой жизни населения на основе удобной и доступной инфраструктуры, благоприятной для проживания окружающей среды
Приоритетное направление 3. Новая индустриализация, переход к цифровой и постиндустриальной экономике
Приоритетное направление 4. Реформирование государственного и муниципального управления, взаимодействия государства, бизнеса и общества, расширение самоуправления и демократизация
Приложение 4. ПЕРЕЧЕНЬ ГОСУДАРСТВЕННЫХ ПРОГРАММ ТАМБОВСКОЙ ОБЛАСТИ (С НАИМЕНОВАНИЕМ КЛЮЧЕВЫХ ПРОЕКТОВ, СООТВЕТСТВУЮЩИХ ПРИОРИТЕТНЫМ НАПРАВЛЕНИЯМ СТРАТЕГИИ СОЦИАЛЬНО-ЭКОНОМИЧЕСКОГО РАЗВИТИЯ ТАМБОВСКОЙ ОБЛАСТИ И ПЛАНИРУЕМЫХ К РЕАЛИЗАЦИИ В РАМКАХ ГОСУДАРСТВЕННЫХ ПРОГРАММ ОБЛАСТИ)

'''

In [38]:
#!g1.1
prompt = text

prompt += "\n\nUser: Есть ли в данном тексте раздел введение? \nAssistant:"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=100,
        eos_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


Введение
1. Краткая характеристика Тамбовской области. Оценка достигнутых целей и анализ потенциала социально-экономического развития Тамбовской области
1.1. Краткая характеристика Тамбовской области
1.2. Оценка достигнутых целей и анализ потенциала социально-экономического развития Тамбовской области
2. Стратегическая цель и приоритеты долгосрочного развития Тамбовской области
3. Целевой сценарий развития Тамбовской области
4. Сроки и этапы реализации Стратегии, показатели достижения целей и ожидаемые результаты реализации Стратегии
5. Стратегические направления и задачи развития Тамбовской области
5.1. Социальная модернизация на базе создания конкурентной среды, инноваций и цифровизации социальной сферы
5.2. Обеспечение комфортной и здоровой жизни населения на основе удобной и доступной инфраструктуры, благоприятной для проживания окружающей среды
5.3. Новая индустриализация, переход к цифровой и постиндустриальной экономике
5.4. Реформирование государственного и муниципального упра

In [39]:
#!g1.1
#!g1.1
# Пример использования:
text = '''
Основные положения
1. Анализ социально-экономического развития Тюменской области
1.1. Оценка достигнутых целей и уровня социально-экономического развития Тюменской области
1.2. Анализ внутренних и внешних факторов, возможностей и ограничений развития Тюменской области
2. Сценарии развития Тюменской области
Консервативный сценарий
Базовый сценарий
3. Приоритеты социально-экономического развития Тюменской области
3.1. Стратегическое видение социально-экономического развития Тюменской области
3.2. Стратегические приоритеты, цели, задачи и направления социально-экономической политики Тюменской области
4. Механизмы реализации Стратегии
5. Оценка финансовых ресурсов для реализации Стратегии
Таблица 11 - Расходы консолидированного бюджета Тюменской области на реализацию мероприятий первого этапа Стратегии (2020 - 2022 годы), млрд рублей
Таблица 12 - Расходы консолидированного бюджета Тюменской области на реализацию мероприятий второго этапа Стратегии (2023 - 2025 годы), млрд рублей
Таблица 13 - Расходы консолидированного бюджета Тюменской области на реализацию мероприятий третьего этапа Стратегии (2026 - 2030 годы), млрд рублей
6. Ожидаемые результаты реализации Стратегии
6.1. Этапы и ожидаемые результаты реализации Стратегии
6.2. Показатели достижения целей социально-экономического развития Тюменской области
Приложение 1. Таблица - Основные показатели социально-экономического развития Тюменской области
Приложение 2. Таблица - Типологизация и определение интересов субъектов стратегического развития Тюменской области
Приложение 3. Таблица - Перечень государственных программ Тюменской области
Приложение 4. ИНВЕСТИЦИОННАЯ СТРАТЕГИЯ ТЮМЕНСКОЙ ОБЛАСТИ
Цели, задачи и результаты реализации Инвестиционной стратегии
Сценарии развития Инвестиционной стратегии
Приоритетные направления улучшения инвестиционного климата
Ключевые инвестиционные проекты Тюменской области
Приложение 5. ИНВЕСТИЦИОННЫЕ ПРОЕКТЫ ТЮМЕНСКОЙ ОБЛАСТИ (ПО СОСТОЯНИЮ НА 17.01.2020, ИСТОЧНИК ИНФОРМАЦИИ - РЕЕСТР ИНВЕСТИЦИОННЫХ, ИННОВАЦИОННЫХ ПРОЕКТОВ И ИНФРАСТРУКТУРНЫХ ПЛОЩАДОК ТЮМЕНСКОЙ ОБЛАСТИ, ЗАКЛЮЧЕННЫЕ КОНЦЕССИОННЫЕ СОГЛАШЕНИЯ)
Таблица 1 - Проекты в стадии реализации (с объемами инвестиций более 100 млн рублей)
Таблица 2 - Планируемые к реализации и перспективные проекты (с объемами инвестиций более 100 млн рублей)
Приложение 6. СТРАТЕГИЯ В ОБЛАСТИ ОБЕСПЕЧЕНИЯ БЛАГОПРИЯТНЫХ УСЛОВИЙ ДЛЯ ВЕДЕНИЯ ЭКСПОРТНОЙ ДЕЯТЕЛЬНОСТИ
1. Текущее состояние экспортной деятельности
2. Анализ потенциально-перспективных экспортных направлений
3. Анализ основных внутренних и внешних барьеров, с которыми сталкиваются экспортеры. Мероприятия по устранению выявленных барьеров
4. Цель и задачи в части развития экспорта
5. Формат и принцип взаимодействия органов исполнительной власти с экспортерами региона, отраслевыми ассоциациями экспортеров
6. Механизм реализации политики в области обеспечения благоприятных условий для ведения экспортной деятельности
7. Ресурсы, необходимые для реализации Экспортной стратегии
8. Ожидаемые результаты в части обеспечения благоприятных условий для ведения экспортной деятельности к 2030 году
Приложение 7. ДОКУМЕНТЫ СТРАТЕГИЧЕСКОГО ПЛАНИРОВАНИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ И ДРУГИЕ НОРМАТИВНЫЕ ПРАВОВЫЕ И ПРАВОВЫЕ АКТЫ РОССИЙСКОЙ ФЕДЕРАЦИИ, УЧТЕННЫЕ ПРИ РАЗРАБОТКЕ СТРАТЕГИИ

'''

In [40]:
#!g1.1
#!g1.1
prompt = text

prompt += "\n\nUser: Есть ли в данном тексте раздел с целевым сценарием социально-экономического развития? \nAssistant:"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=100,
        eos_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


Основные положения
1. Анализ социально-экономического развития Тюменской области
1.1. Оценка достигнутых целей и уровня социально-экономического развития Тюменской области
1.2. Анализ внутренних и внешних факторов, возможностей и ограничений развития Тюменской области
2. Сценарии развития Тюменской области
Консервативный сценарий
Базовый сценарий
3. Приоритеты социально-экономического развития Тюменской области
3.1. Стратегическое видение социально-экономического развития Тюменской области
3.2. Стратегические приоритеты, цели, задачи и направления социально-экономической политики Тюменской области
4. Механизмы реализации Стратегии
5. Оценка финансовых ресурсов для реализации Стратегии
Таблица 11 - Расходы консолидированного бюджета Тюменской области на реализацию мероприятий первого этапа Стратегии (2020 - 2022 годы), млрд рублей
Таблица 12 - Расходы консолидированного бюджета Тюменской области на реализацию мероприятий второго этапа Стратегии (2023 - 2025 годы), млрд рублей
Таблица 1

In [ ]:
#!g1.1
